In [2]:
import os
import requests

gitIGNORE = 'gitIGNORE'
nasa_api = f'nasa-api.txt'

def createMyFolders(folder_names_list: list, destination: str):
    """
    desitination: absolute path
    """
    import os

    if not os.path.exists(destination): # check if destination exists
        print(f"Destination does NOT exists: {destination}")
        return
    
    # loop thru all folder names and attempt to create them
    for folder_name in folder_names_list:
        folder_path = os.path.join(destination, folder_name)
        try:
            os.makedirs(folder_path, exist_ok=True)
            print(f"Created folder: {folder_path}")
        except Exception as e:
            print(f"Failed to create folder {folder_path}: {e}")

    return None

createMyFolders([gitIGNORE], os.getcwd()) # create folders in the current working directory

# open(f'{gitIGNORE}/{nasa_api}', 'a').close() # open an empty file in append mode and close it so empty file is created



Created folder: c:\NASA-open\gitIGNORE


In [3]:
with open(rf"{gitIGNORE}\{nasa_api}", 'r') as file:
    apikey = file.read()

In [11]:
baseURL = 'https://api.nasa.gov/planetary/apod'

params = {
    'api_key': apikey,
    # 'date': '2025-08-01',  
    'hd': True              # Request high-definition image (if available)
}

response = requests.get(baseURL, params=params)

if response.status_code == 200:
    data = response.json()
    print(f"Title: {data['title']}")
    print(f"Date: {data['date']}")
    # print(f"Explanation: {data['explanation']}")
    exp = data['explanation'].split('. ')
    [print(f'{_}. ') for _ in exp]  
    print(f"File Type: {data['media_type']}")
    print(f"Image URL: {data['hdurl'] if 'hdurl' in data else data['url']}")
else:
    print(f"Error {response.status_code}: {response.text}")


Title: Milky Way and Exploding Meteor
Date: 2025-08-03
In about a week the Perseid Meteor Shower will reach its maximum. 
 Grains of icy rock will streak across the sky as they evaporate during entry into Earth's atmosphere. 
 These grains were shed from Comet Swift-Tuttle. 
 The Perseids result from the annual crossing of the Earth through Comet Swift-Tuttle's orbit, and are typically the most active meteor shower of the year. 
 Although it is hard to predict the level of activity in any meteor shower, in a clear dark sky an observer might see a meteor a minute. 
 This year's Perseids peak just a few days after  full moon, and so some faint meteors will be lost to the lunar skyglow. 
 Meteor showers in general are best seen from a relaxing position, away from lights. 
 Featured here is a meteor caught exploding during the 2015 Perseids above Austria next to the central band of our Milky Way Galaxy.. 
File Type: image
Image URL: https://apod.nasa.gov/apod/image/2508/MeteorBoom_vanderHo